In [1]:
import numpy as np

In [45]:
M = np.asarray(np.asmatrix('0 0 3 1;0 0 1 3; 1 3 0 0;3 1 0 0'))
q = np.asarray([-1, -1, -1, -1], dtype=np.float64)

#M = np.asarray(np.asmatrix('0.2100 0.1797; 0.1797 0.8866'))
#q = np.asarray([-0.1524, -0.4427], dtype=np.float64)

In [125]:
def lcpipm(M, x, s, epsilon, tau, theta):
    i = 0
    j = 0
    
    n = M.shape[0]

    mu = np.dot(x.T, s)/n
    v = np.sqrt((x.T*s)/mu)
    
    display(kernel_func(v))
        
    bm = np.zeros(shape=(n + n, n + n))
    
    bm[:n, :n] = -M
    bm[:n, n:] = np.eye(n)

    rhs = np.zeros(shape=(n + n)).T
    
    while mu*n >= epsilon:
        i += 1
        mu = (1 - theta)*mu;
        v = np.sqrt((x.T*s)/mu)
        #p = kernel_func(v)
        #while p > tau and j < 50:
        #j += 1
        bm[n:, :n] = np.diag(s)
        bm[n:, n:] = np.diag(x)

        rhs[n:] = -mu*v*kernel_func_dv(v)

        xs = np.linalg.solve(bm, rhs)
        delta_x = xs[:n]
        delta_s = xs[n:]

        alpha_x, alpha_s, alpha_k = get_steplength(x, s, delta_x, delta_s, 0.85, n)

        x += alpha_k*delta_x
        s += alpha_k*delta_s

        #v = np.sqrt((x.T*s)/mu)
        #p = kernel_func(v)    
            
    return x, s, i


def get_steplength(x, s, delta_x, delta_s, eta, n):
    alpha_x = 1.0
    alpha_s = 1.0
    for i in range(n):
        if delta_x[i] < 0:
            # get alpha primal
            alpha_x = min(alpha_x, -x[i] / delta_x[i])
        if delta_s[i] < 0:
            # get alpha dual
            alpha_s = min(alpha_s, -s[i] / delta_s[i])
    # choose smallest alpha
    alpha_x = min(1.0, eta * alpha_x)
    alpha_s = min(1.0, eta * alpha_s)
    alpha_k = min(1.0, eta * min(alpha_s, alpha_x))

    return alpha_x, alpha_s, alpha_k


def kernel_func(v):
    return sum(((np.power(v, 2) - 1)/2)+ 2*np.log(1 + np.reciprocal(v)) - 2*np.log(2))

def kernel_func_dv(v):
    return sum((v - (2/(np.power(v, 2) + v))))

In [132]:
epsilon = 10e-6
tau = 3
theta = 0.3

x0 = np.asarray([4, 4, 4, 4], dtype=np.float64)
s0 = np.dot(M, x0) + q
#s0 = np.asarray([0.0782, 0.4427], dtype=np.float64)

results = lcpipm(M=M, x=x0, s=s0, epsilon=epsilon, tau=tau, theta=theta)
display(results)
# display('x : [{:.6f}, {:.6f}]'.format(results[0][0], results[0][1]))
# display('s : [{:.6f}, {:.6f}]'.format(results[1][0], results[1][1]))
# display('Total Iterations : {}'.format(results[2]))

0.0

(array([0.25000517, 0.25000517, 0.25000517, 0.25000517]),
 array([2.06872256e-05, 2.06872247e-05, 2.06872287e-05, 2.06872216e-05]),
 48)

In [ ]:
test_v = np.asarray([3, 3], dtype=np.float64)
display(test_v)
display(np.power(test_v, 2))
display((np.power(test_v, 2) - 1)/2)
display(1 + np.reciprocal(test_v))
display(np.log(1 + np.reciprocal(test_v)))

kernel_func(test_v)

In [ ]:
x0 = np.asarray([3, 3], dtype=np.float64)
s0 = np.dot(M, x0) + q